# Multivariate linear regression

TODO:

In [1]:
import numpy as np
import pandas as pd

In [16]:
CSV_PATH = "https://gitlab.com/oasci/courses/pitt/biosc1540-2024s/-/raw/main/biosc1540/files/csv/advertising-data.csv"

df = pd.read_csv(CSV_PATH)

In [19]:
label_column = "Product_Sold"

df_labels = df[label_column]
df_features = df.drop(columns=[label_column], inplace=False)

labels = df_labels.to_numpy().reshape(-1, 1)
features = df_features.to_numpy()

## Linear

In [27]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X=features, y=labels)
print(reg.coef_)
print(reg.intercept_)
print(reg.score(X=features, y=labels))

[[1.97147823 2.79786525 1.59446751 2.43283307 1.40693022 3.91183385]]
[36.65524744]
0.9401750192922066


## Ridge

In [28]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=1.0)
reg.fit(X=features, y=labels)
print(reg.coef_)
print(reg.intercept_)
print(reg.score(X=features, y=labels))

[[1.97147817 2.79786515 1.59446745 2.43283298 1.40693016 3.9118337 ]]
[36.65551041]
0.9401750192922053


## Acknowledgements

Much of this material has been adapted from [An Introduction to Statistical Learning](https://www.statlearning.com/) to be more digestible for undergraduates starting in Biology.